In [61]:
import numpy as np

## Задание 2

In [62]:
A = np.array([
        [-2, -1, 1, -7, 1, 0, 0, 2],
        [-4, 2, 1, 0, 5, 1, -1, 5],
        [1, 1, 0, 1, 4, 3, 1, 1]
    ])

b = [-2, 8, -2]
c = [12, -2, -6, 20, -18, -5, -7, -20]
y_0 = [-3, -2, -1]
J_B = [2, 4, 6]
delta = np.dot(y_0, A) - c
m, n = A.shape
optimal_solution = None

In [63]:
class OptFounException(Exception):
    def __init__(self, opt_x):
        self.opt_x = opt_x
        
        
        
class IncompatibleRestrictionsException(Exception):
    pass

In [64]:
def find_get_el_ind(arr, el):
    for i in range(len(arr)):
        if arr[i] == el:
            return i
        
    return -1


def get_sigma_da(mu, delta, J_N):
    sigma = [math.inf] * len(delta)
    
    if np.all(mu[J_N] >= 0):
        raise IncompatibleRestrictionsException
    
    
    for i in J_N:
        if mu[i] < 0:
            sigma[i] = -delta[i] / mu[i]
    
    
    min_sigma = min(sigma)
    return min_sigma, np.where(sigma == min(sigma))[0][0]


def get_J_N(J_B): return list(set(range(n)) - set(J_B))


def check_optimality(y_0, chi_B):
    if len(np.where(chi_B < 0)[0]) == 0:
        optimal_solution = chi_B
        raise OptFounException(chi_B)

In [65]:
def dual_sm_iteration(y_0, J_B, delta):
    print(f'y_0: {y_0}')
    J_N = get_J_N(J_B)
    A_B = A[:, J_B]
    B = np.linalg.inv(A_B)
    chi_B = np.dot(B, b)
    
    check_optimality(y_0, chi_B)
    
    k = np.where(chi_B < 0)[0][0]
    j_k =  J_B[k] # ind_B for which chi_i < 0
    mu = np.dot(B[k, :], A)
    
    print(f'J_B: {J_B}')
    print(f'A_B: \n {A_B}', end='\n\n')
    print(f'B: \n {B}', end='\n\n')
    print(f'k: {k}')
    print(f'j_k: {j_k}')
    print(f'mu: {mu}')
    
    sigma_0, new_j_k = get_sigma_da(mu, delta, J_N)
    y_new = y_0 + sigma_0 * np.dot(mu[J_B], B)
    delta_new = delta + sigma_0 * mu
    J_B_new = list(set(J_B) - set([j_k])) + [new_j_k]
    
    print(f'sigma_0: {sigma_0}')
    print(f'y_new: {y_new}')
    print(f'delta_new: {delta_new}')
    print(f'J_B_new: {J_B_new}')
    print('_' * 100)
    return y_new, J_B_new, delta_new

### Итерации алгоритма

In [66]:
new_y, new_J_B, new_delta = dual_sm_iteration(y_0, J_B, delta)
try:
    while True: 
        new_y, new_J_B, new_delta = dual_sm_iteration(new_y, new_J_B, new_delta)
except OptFounException as e:
    print(e.opt_x)
except IncompatibleRestrictionsException:
    print('Restrictions are incompatible')
    

y_0: [-3, -2, -1]
J_B: [2, 4, 6]
A_B: 
 [[ 1  1  0]
 [ 1  5 -1]
 [ 0  4  1]]

B: 
 [[ 1.125 -0.125 -0.125]
 [-0.125  0.125  0.125]
 [ 0.5   -0.5    0.5  ]]

k: 0
j_k: 2
mu: [-1.875 -1.5    1.    -8.     0.    -0.5    0.     1.5  ]
sigma_0: -0.0
y_new: [-3. -2. -1.]
delta_new: [1. 0. 1. 0. 1. 0. 8. 3.]
J_B_new: [4, 6, 1]
____________________________________________________________________________________________________
y_0: [-3. -2. -1.]
J_B: [4, 6, 1]
A_B: 
 [[ 1  0 -1]
 [ 5 -1  2]
 [ 4  1  1]]

B: 
 [[ 0.25        0.08333333  0.08333333]
 [-0.25       -0.41666667  0.58333333]
 [-0.75        0.08333333  0.08333333]]

k: 1
j_k: 6
mu: [ 2.75        0.         -0.66666667  2.33333333  0.          1.33333333
  1.         -2.        ]
sigma_0: 1.4999999999999998
y_new: [-3.375 -2.625 -0.125]
delta_new: [5.1250000e+00 0.0000000e+00 0.0000000e+00 3.5000000e+00 1.0000000e+00
 2.0000000e+00 9.5000000e+00 4.4408921e-16]
J_B_new: [1, 4, 2]
________________________________________________________

### Задание 3

In [67]:
c = np.array([1, 4, -9, 6, -5, 8, 3, -7, 1])
b = np.array([14, 23, 6])
A = np.array([
        [0, 1, -4, 5, 0, -3, -5, 0, 1],
        [1, -1, 0, 1, 0, 8, 1, -2, 1],
        [0, 6, -1, 0, -2, 3, 8, 1, 1]
    ]
)
x_0 = np.array([3, 0, 0, 4, 0, 2, 0, 0, 0])
J_B = np.array([0, 3, 5])
u_0 = np.array([1, 1, 1])

In [68]:
def get_phi_sense(z, x_B):
    phi = [math.inf] * len(z)
    for i in range(len(z)):
        if z[i] < 0:
            phi[i] = - x_B[i] / z[i]
            
    return phi


def get_new_x_0_sense(x_0, phi_0, z, J_B, j_s):
    new_x_0 = np.array(x_0)
    new_x_0[j_s] = 0
    for i in range(len(J_B)):
        ind = J_B[i]
        x_0[ind] += phi_0 * z[i]
        
    return new_x_0


In [69]:
J_N = get_J_N(J_B)
A_B = A[:, J_B]
A_new = A[[0, 1]]
z = np.dot(np.linalg.inv(A_B), [0, 0, 1])
j_s = J_B[np.where(z < 0)[0][0]]
phi = get_phi_sense(z, x_0[J_B])
phi_0 = min(phi)
new_x_0 = get_new_x_0_sense(x_0, phi_0, z, J_B, j_s)


print(f'z: {z}')
print(f'j_s: {j_s}')
print(f'phi: {phi}')
print(f'phi_0: {phi_0}')
print(f'new_x_0: {new_x_0}')

z: [-2.86666667  0.2         0.33333333]
j_s: 0
phi: [1.0465116279069768, inf, inf]
phi_0: 1.0465116279069768
new_x_0: [0 0 0 4 0 2 0 0 0]


In [53]:
new_x_0

array([0, 0, 0, 4, 0, 2, 0, 0, 0])